In [57]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from RNN import RNN
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
url = "https://github.com/swarj/MLProject/raw/main/apple_share_price.csv" #get the dataset
raw_df = pd.read_csv(url)
raw_df = raw_df.loc[::-1] #reverse order of rows to chronologically make it go from past --> present
raw_df = raw_df.assign(Time=range(len(raw_df)))
df = raw_df.reindex(['Time','Date', 'High', 'Low', 'Volume', 'Open', 'Close'], axis = 1) #make target variable last column, if we want close instead of open we can switch it
#we can try to predict the close value for the following day, need to add target close value  
df['TargetNextClose'] = df['Close'].shift(-1)
df.drop(df.tail(1).index,inplace=True) # drop last rows
df.style
print(df.head(10))
print(df.shape)
df.reset_index(inplace=True)
data_set = df.iloc[:, 0:8]
pd.set_option('display.max_columns', None)
print(data_set.head(10))
data_set = data_set.to_numpy()

#(Jason) i added this for my forward method
# Parse the date column as a datetime object
df['Date'] = pd.to_datetime(df['Date'])
# Convert the date column to a float timestamp
df['Date'] = df['Date'].apply(lambda x: x.timestamp())
# Reset the index
df.reset_index(inplace=True)
# Select the first 8 columns as the dataset
data_set = df.iloc[:, 0:8].to_numpy()
# Print the first 10 rows of the updated dataframe
print(df.head(10))

      Time       Date   High    Low     Volume   Open  Close  TargetNextClose
1663     0   3-Jan-11  47.18  46.41  111280407  46.52  47.08            47.33
1662     1   4-Jan-11  47.50  46.88   77337001  47.49  47.33            47.71
1661     2   5-Jan-11  47.76  47.07   63879193  47.08  47.71            47.68
1660     3   6-Jan-11  47.89  47.56   75106626  47.82  47.68            48.02
1659     4   7-Jan-11  48.05  47.41   77982212  47.71  48.02            48.92
1658     5  10-Jan-11  49.03  48.17  112139482  48.40  48.92            48.81
1657     6  11-Jan-11  49.28  48.50  111019993  49.27  48.81            49.20
1656     7  12-Jan-11  49.20  48.86   75644310  49.04  49.20            49.38
1655     8  13-Jan-11  49.52  49.12   74536182  49.31  49.38            49.78
1654     9  14-Jan-11  49.78  49.21   77209748  49.41  49.78            48.66
(1663, 8)
   index  Time       Date   High    Low     Volume   Open  Close
0   1663     0   3-Jan-11  47.18  46.41  111280407  46.52  47.08
1 

In [59]:
X = []
backcandles = 30
#print(type(data_set))
#print(data_set.shape[0])
#print(data_set[40-backcandles:40, 5])
for j in range(8):
    X.append([])
    for i in range(backcandles, data_set.shape[0]):
        X[j].append(data_set[i-backcandles:i, j])

X=np.moveaxis(X, [0], [2])

X, yi =np.array(X), np.array(data_set[backcandles:,-1])
y=np.reshape(yi,(len(yi),1))
print(y.shape)
print(yi)


(1633, 1)
[ 51.31  51.54  51.04 ... 158.6  159.26 159.9 ]


In [60]:
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(type(y_train))



1306
<class 'numpy.ndarray'>


In [61]:
# Create an instance 
rnn = RNN(input_size=2, output_size=1, hidden_size=3)
# Define some inputs
input_data = [[1, 0], [0, 1], [0.2550690257394217, 0.49543508709194095], [0, 0]]
# Call the forward method and print the outputs
outputs = rnn.forward_pass(input_data)
print(outputs)

[array([[0.5903899]]), array([[1.04915395]]), array([[1.17997152]]), array([[1.20980181]])]


In [62]:
rnn = RNN(8,1,2)
rnn.train(X_train, y_train)

c:\Users\namho\Downloads\ML_proj\MLProject\RNN.py:4: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


AttributeError: 'list' object has no attribute 'shape'